In [1]:
import pandas as pd
import matplotlib as plt

In [2]:
# Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:

# InvoiceNo — номер транзакции
# StockCode — код товара
# Description — описание товара
# Quantity — количество единиц товара, добавленных в заказ
# InvoiceDate — дата транзакции 
# UnitPrice — цена за единицу товара
# CustomerID — id клиента
# Country — страна, где проживает клиент

In [3]:
path_to_file = '5_data.csv.zip'

In [4]:
retail = pd.read_csv(path_to_file, compression='zip', encoding='ISO-8859-1')
retail.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


In [5]:
retail_columns = retail.columns


In [6]:
retail.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [7]:
mask_duplicate = retail.duplicated(subset=retail_columns, keep='first')
mask_duplicate[mask_duplicate == True].count()
retail = retail[~mask_duplicate]

In [8]:
count = 0
retail.InvoiceNo[[i.startswith('C') for i in retail.InvoiceNo]].count()

9251

In [9]:
retail = retail.query('Quantity > 0')
retail.shape


(526054, 8)

In [10]:
retail_top_purchases = retail\
    .query("Country == 'Germany'")\
    .groupby(['CustomerID'], as_index=False)\
    .agg({'InvoiceNo':'nunique'})\
    .rename(columns={'InvoiceNo':'count_purchases'})\
    .sort_values(by='count_purchases', ascending=False)
retail_top_purchases

,CustomerID,count_purchases
37,12569.0,32
3,12471.0,30
83,12720.0,25
52,12621.0,20
6,12474.0,18
...,...,...
62,12644.0,1
48,12612.0,1
50,12614.0,1
53,12622.0,1


In [11]:
quantile_80_purchases = retail_top_purchases.count_purchases.quantile(q=0.8)
quantile_80_purchases

7.0

In [12]:
germany_top = retail_top_purchases.CustomerID[retail_top_purchases.count_purchases > quantile_80_purchases]

In [13]:
germany_top

37    12569.0
3     12471.0
83    12720.0
52    12621.0
6     12474.0
79    12709.0
81    12712.0
64    12647.0
16    12500.0
69    12662.0
8     12476.0
12    12481.0
44    12600.0
78    12708.0
56    12626.0
77    12705.0
51    12619.0
26    12524.0
Name: CustomerID, dtype: float64

In [14]:
top_retail_germany = retail[retail.CustomerID.isin(germany_top)]

In [16]:
top_retail_germany.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany


In [21]:
top_retail_germany.query("StockCode != 'POST'").groupby('StockCode').agg({'InvoiceNo':'count'}).sort_values(by='InvoiceNo')

,InvoiceNo
StockCode,
10125,1
22246,1
23176,1
23179,1
22242,1
...,...
22961,28
20719,30
22423,34


In [ ]:
# Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, 
#используя колонки Quantity и UnitPrice.

# Отфильтрованные данные записаны в retail.

In [23]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

In [28]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [29]:
# Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. 
# В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа 
# (через запятую с пробелом и в порядке убывания TotalRevenue)

In [35]:
top_5_retail = retail.groupby('InvoiceNo', as_index=False).agg({'Revenue':'sum'}).sort_values(by='Revenue', ascending=False).head(5)

In [42]:
for i in top_5_retail['InvoiceNo'].to_list():
    print(i + ',')

581483,
541431,
574941,
576365,
556444,
